In [1]:
import numpy as np
import pandas as pd

In [2]:
def aggregateCluster(clusterNum,ptype):
    GLList=[]
    if ptype =='Batters':
        Clusters = pd.read_csv('./BatterClusters/PredBattingClusters.csv', index_col=0)
    else:
        Clusters = pd.read_csv('./PitcherClusters/PredPitchingClusters.csv', index_col=0)
    
    PlayersCluster = Clusters[Clusters['Labels']==clusterNum]
    for players in PlayersCluster.values:
        ID=players[1]
        if players[3] == 'Y':
            GameLog = pd.read_csv('../Data/DataBaseCrawler/GameLogs/'+ptype+'/'+str(ID)+'GL.csv', index_col=0)
        else:
            GameLog = pd.read_csv('../Data/DataBaseCrawler/GameLogs/Inactive'+ptype+'/'+str(ID)+'GL.csv', index_col=0)
        GameLog = GameLog[GameLog['Date']!='Total']
        GLList.append(GameLog)
    return pd.concat(GLList)

In [3]:
def stripAt(x):
    return x.strip('@')

In [4]:
def cleanGL(x):
    try:
        out = float(x)
    except:
        try:
            out = float(x.strip('%'))
        except:
            out = 0
    return out

In [5]:
#Get The batter cluster types
BattingClusters = pd.read_csv('./BatterClusters/PredBattingClusters.csv', index_col=0)
PitchingClusters = pd.read_csv('./PitcherClusters/TotalPitchingClusters.csv', index_col=0)

In [6]:
PitchingClusters[PitchingClusters['Name']=='Mike Montgomery']

,Name,ID,Labels,Active


In [7]:
#Test the function
PitcherCluster=aggregateCluster(6,'Pitchers')
PitcherCluster=PitcherCluster[PitcherCluster['GS']==1]
PitcherCluster['Opp']=PitcherCluster['Opp'].apply(stripAt)
PitcherCluster.shape

(7566, 42)

In [8]:
BatterCluster = aggregateCluster(17,'Batters')
BatterCluster = BatterCluster[~BatterCluster['Pos'].isin(['PR-LF', 'PH-CF', 'PH-RF','PH-1B','PH-SS','PH-2B','PR-C','P','PH-RF-CF','PR-DH','PH-3B','PH-C','PH','PH-LF','DH','PR-1B','PR-RF','PH-DH','1B-PH','RF-PH'])]
BatterCluster = BatterCluster[BatterCluster['BO']!=0]
BatterCluster['Opp']=BatterCluster['Opp'].apply(stripAt)
BatterCluster.shape

(12303, 40)

In [9]:
MergedClusters=pd.merge(BatterCluster,PitcherCluster, left_on=['Date','Team','Opp'], right_on=['Date','Opp','Team'])
MergedClusters=MergedClusters[MergedClusters['PA']!=0]
MergedClusters.columns

Index(['Date', 'Team_x', 'Opp_x', 'BO', 'Pos', 'G_x', 'AB', 'PA', 'H_x', '1B',
       '2B', '3B', 'HR_x', 'R_x', 'RBI', 'BB_x', 'IBB_x', 'SO_x', 'HBP_x',
       'SF', 'SH', 'GDP', 'SB', 'CS', 'AVG_x', 'DKfp_x', 'BB%_x', 'K%_x',
       'BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'Spd', 'BABIP_x', 'wSB', 'wRC',
       'wRAA', 'wOBA', 'wRC+', 'Team_y', 'Opp_y', 'GS', 'W', 'L', 'ERA', 'G_y',
       'GS_x', 'CG', 'ShO', 'SV', 'HLD', 'BS', 'IP', 'TBF', 'H_y', 'R_y', 'ER',
       'HR_y', 'BB_y', 'IBB_y', 'HBP_y', 'WP', 'BK', 'SO_y', 'GSv2', 'DKfp_y',
       'K/9', 'BB/9', 'K/BB', 'HR/9', 'K%_y', 'BB%_y', 'K-BB%', 'AVG_y',
       'WHIP', 'BABIP_y', 'LOB%', 'ERA-', 'FIP-', 'FIP'],
      dtype='object')

In [10]:
MergedClusters=MergedClusters[['BO', 'G_x', 'AB', 'PA', 'H_x', '1B','2B', '3B', 'HR_x', 'R_x', 'RBI', 'BB_x', 'IBB_x', 'SO_x', 'HBP_x','SF', 'SH', 'GDP', 'SB', 'CS', 'AVG_x', 'DKfp_x', 'BB%_x', 'K%_x','BB/K', 'OBP', 'SLG', 'OPS', 'ISO', 'Spd', 'BABIP_x', 'wSB', 'wRC','wRAA', 'wOBA', 'wRC+', 'Team_y', 'Opp_y', 'GS', 'W', 'L', 'ERA', 'G_y',
       'GS_x', 'CG', 'ShO', 'SV', 'HLD', 'BS', 'IP', 'TBF', 'H_y', 'R_y', 'ER',
       'HR_y', 'BB_y', 'IBB_y', 'HBP_y', 'WP', 'BK', 'SO_y', 'GSv2', 'DKfp_y',
       'K/9', 'BB/9', 'K/BB', 'HR/9', 'K%_y', 'BB%_y', 'K-BB%', 'AVG_y',
       'WHIP', 'BABIP_y', 'LOB%', 'ERA-', 'FIP-', 'FIP']].applymap(cleanGL)

In [11]:
MergedClusters.shape

(1048, 77)

In [12]:
ClusterSplit = MergedClusters.sum(axis=0).to_frame().T
ClusterSplit.head()

,BO,G_x,AB,PA,H_x,1B,2B,3B,HR_x,R_x,...,K%_y,BB%_y,K-BB%,AVG_y,WHIP,BABIP_y,LOB%,ERA-,FIP-,FIP
0,6476.0,1048.0,3443.0,3886.0,813.0,529.0,178.0,17.0,89.0,394.0,...,22592.3,9452.4,13143.5,249.028,1480.42,300.027,78482.7,120775.0,107278.0,4400.8


[]